# mastercan to datafreame to compare|

In [ ]:
print('hello')

In [ ]:
! dir D:\fork\lipidxplorer-evaluation\190731_benchmark_data_files_infos\190731_mzML_no_zlib

In [ ]:
left_sc_path = r'D:\fork\lipidxplorer-evaluation\190731_benchmark_data_files_infos\190731_mzML_no_zlib\for_paper_from_df.sc'
right_sc_path = r'D:\fork\lipidxplorer-evaluation\190731_benchmark_data_files_infos\190731_mzML_no_zlib\190731_mzML_no_zlib_128.sc'
import pickle
import pandas as pd
import numpy as np

with open(left_sc_path, 'rb') as fh:
    left_scan = pickle.load(fh)

left_df = pd.DataFrame((se.dictIntensity for se in left_scan.listSurveyEntry))
left_df['precurmass'] = [se.precurmass for se in left_scan.listSurveyEntry]
#se.listMSMS[0].mass, se.listMSMS[0].dictIntensity

left_df.sort_values('precurmass', inplace=True) # for merge_asof
left_df['left_mass'] = left_df['precurmass']

with open(right_sc_path, 'rb') as fh:
    right_scan = pickle.load(fh)

right_df = pd.DataFrame((se.dictIntensity for se in right_scan.listSurveyEntry))
right_df['precurmass'] = [se.precurmass for se in right_scan.listSurveyEntry]
right_df.sort_values('precurmass', inplace=True) # for merge_asof
right_df['right_mass'] = right_df['precurmass']
df = pd.merge_asof(left_df, right_df, on='precurmass',tolerance = 0.01 )

left_cols = left_df.columns[:-2].to_list()
right_cols = right_df.columns[:-2].to_list()

df.replace(0, np.NaN, inplace=True)

In [ ]:

df

In [ ]:
df['mean_left'] = df.replace(0, np.NaN).loc[:,left_cols].mean(axis=1)
df['sum_left'] = df.replace(0, np.NaN).loc[:,left_cols].sum(axis=1)

In [ ]:
df['mean_right'] = df.replace(0, np.NaN).loc[:,right_cols].mean(axis=1)
df['sum_right'] = df.replace(0, np.NaN).loc[:,right_cols].sum(axis=1)

In [ ]:
df['lr_ratio']=df['mean_left']/df['mean_right']
df['mass_diff'] = (df['left_mass']-df['right_mass']).abs()

In [ ]:
interesting_columns = 'left_mass right_mass	mean_left	sum_left	mean_right	sum_right	lr_ratio	mass_diff'.split()

In [ ]:
df.iloc[df['lr_ratio'].nlargest().index][interesting_columns]

In [ ]:
df.iloc[df['lr_ratio'].nsmallest().index][interesting_columns]

In [ ]:
df.loc[594]

In [ ]:
df.loc[592:596][interesting_columns]

# get the og data

In [ ]:
import pickle
with open('spectra_dfs.pkl' ,'rb') as fh:
    spectra_dfs = pickle.load(fh)

In [ ]:
from LX2_masterscan import add_group_no
df = spectra_dfs[2]
ms1_peaks = df.loc[df.precursor_id.isna()]
occupancy=0.5
add_group_no(ms1_peaks, occupancy, cleanup_cols=False)

In [ ]:
ms1_peaks[ms1_peaks.mz.between(577,579)].to_clipboard()

In [ ]:
ms1_peaks[ms1_peaks.group_no== 6947.0].plot.scatter(x='mz', y='inty')